<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/08-%E5%8F%AC%E5%9B%9E%E6%A8%A1%E5%9E%8B/00_ComiRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComiRec

论文：Controllable Multi-Interest Framework for Recommendation
很多序列化推荐方法都最终产生一个user emb，去item emb空间中检索出最相关的item；而user在一段时间内，是有多种兴趣的，应该要映射到多个emb去检索。

如何获取这些兴趣向量：动态路由、自注意力机制；
需要：seq_id,target_id,label；序列如何构建？

基于seq_id，得到多个兴趣向量表示用户（之前是1个向量表示）；每一个兴趣向量，与候选item进行向量检索，取softmax概率最大的item，作为该兴趣命中。最终得到四个兴趣对应的四个视频的向量，作为user_emb



In [ ]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D

## 0.准备工作

In [ ]:
seq = fc.sequence_categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.sequence_categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_col = fc.embedding_column(seq, dimension=8)
target_col = fc.embedding_column(target, dimension=8)
label = fc.sequence_numeric_column('label', dtype=tf.float32)
columns = [seq_col, target_col, label]
features={
  "seq": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[1100, 1101, 1102, 1101, 1103],
      dense_shape=[3, 2]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),
  "label": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1.0,0,1],
      dense_shape=[3, 1]),
}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

In [ ]:
sequence_feature_layer = SequenceFeatures(columns, name='sequence_features_input_layer')
seq_input, seq_len = sequence_feature_layer(features)
seq_input.keys()

dict_keys(['label', 'seq_embedding', 'target_embedding'])

In [ ]:
seq_mask, label = seq_len['seq_embedding'], seq_len['label']
seq_mask, label

(<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 2, 1])>,
 <tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 1, 1])>)

In [ ]:
item_list_emb, label, target_emb= seq_input['seq_embedding'], seq_input['label'], seq_input['target_embedding']
item_list_emb, target_emb, label

(<tf.Tensor: shape=(3, 2, 8), dtype=float32, numpy=
 array([[[ 0.6704128 , -0.4825759 ,  0.41079706, -0.05277091,
          -0.18126296,  0.08896871, -0.01469795,  0.29657248],
         [-0.10857867, -0.06337585, -0.16433023, -0.13755628,
          -0.0994873 , -0.02037037,  0.28568947,  0.2076284 ]],
 
        [[ 0.15900598,  0.54094213, -0.15709746, -0.40975308,
          -0.39427465,  0.12961422,  0.30570927,  0.15592112],
         [-0.10857867, -0.06337585, -0.16433023, -0.13755628,
          -0.0994873 , -0.02037037,  0.28568947,  0.2076284 ]],
 
        [[-0.30083585,  0.35435283,  0.2086421 , -0.07965193,
          -0.27387637,  0.50393224, -0.3017337 , -0.05612677],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ]]],
       dtype=float32)>, <tf.Tensor: shape=(3, 1, 8), dtype=float32, numpy=
 array([[[ 0.2682965 ,  0.444604  , -0.34399304, -0.1467334 ,
          -0.02334886,  0.17901677, -0.13051414,  0.0

## 1.位置编码

In [ ]:
max_len = tf.shape(item_list_emb)[1]
emb_dims = 8
position_embedding = tf.compat.v1.get_variable(shape=[1, max_len, emb_dims],name='position_embedding')
item_list_add_pos = item_list_emb + tf.tile(position_embedding, [tf.shape(item_list_emb)[0], 1, 1])
item_list_add_pos

<tf.Tensor: shape=(3, 2, 8), dtype=float32, numpy=
array([[[ 0.27248767, -0.347915  , -0.08105707, -0.3131885 ,
         -0.8505138 ,  0.25491077, -0.5917015 , -0.25108126],
        [ 0.08465088, -0.44792396, -0.57523763, -0.7947656 ,
         -0.7840925 ,  0.64095175,  0.18166947,  0.04904279]],

       [[-0.23891912,  0.67560303, -0.6489516 , -0.67017066,
         -1.0635254 ,  0.2955563 , -0.27129427, -0.39173263],
        [ 0.08465088, -0.44792396, -0.57523763, -0.7947656 ,
         -0.7840925 ,  0.64095175,  0.18166947,  0.04904279]],

       [[-0.698761  ,  0.48901373, -0.28321204, -0.3400695 ,
         -0.9431272 ,  0.6698743 , -0.8787372 , -0.6037805 ],
        [ 0.19322956, -0.38454813, -0.41090742, -0.6572093 ,
         -0.6846052 ,  0.6613221 , -0.10402   , -0.15858561]]],
      dtype=float32)>

## 2.multi_head_att

In [ ]:
num_heads = num_interest = 4
hidden_size = 4
mha = Dense(hidden_size * num_heads, activation=tf.nn.tanh)
att_w  = Dense(num_heads, activation=None)

item_hidden = mha(item_list_add_pos)
item_att_w  = att_w(item_hidden)
item_att_w = tf.transpose(item_att_w, [0,2,1])
item_att_w

<tf.Tensor: shape=(3, 4, 2), dtype=float32, numpy=
array([[[ 0.15945143, -0.30868155],
        [-0.44137686, -0.407699  ],
        [-0.19866277,  0.08905888],
        [ 0.41580737, -0.3413364 ]],

       [[-0.32409707, -0.30868155],
        [-1.1242661 , -0.407699  ],
        [ 0.14833081,  0.08905888],
        [ 0.14246675, -0.3413364 ]],

       [[-0.39153647, -0.30699012],
        [-0.8790223 , -0.40335512],
        [ 0.10243495,  0.02444693],
        [ 0.25796926, -0.20628071]]], dtype=float32)>

## 2.1计算SA

In [ ]:
seq_mask_ = tf.expand_dims(tf.where(tf.sequence_mask(seq_mask),1.0,0.0),axis=1)
atten_mask = tf.tile(seq_mask_, [1, num_heads, 1])
paddings = tf.ones_like(atten_mask) * (-2 ** 32 + 1)
item_att_w = tf.where(tf.equal(atten_mask, 0), paddings, item_att_w)
item_att_w = tf.nn.softmax(item_att_w)
item_att_w

<tf.Tensor: shape=(3, 4, 2), dtype=float32, numpy=
array([[[0.6149418 , 0.38505825],
        [0.49158132, 0.5084187 ],
        [0.42856172, 0.57143825],
        [0.68073326, 0.3192667 ]],

       [[0.49614623, 0.50385386],
        [0.32814935, 0.6718506 ],
        [0.5148136 , 0.4851863 ],
        [0.6186455 , 0.38135442]],

       [[1.        , 0.        ],
        [1.        , 0.        ],
        [1.        , 0.        ],
        [1.        , 0.        ]]], dtype=float32)>

In [ ]:
# 用户的4个兴趣向量
interest_emb = tf.matmul(item_att_w, item_list_emb)
user_eb = interest_emb
user_eb

<tf.Tensor: shape=(3, 4, 8), dtype=float32, numpy=
array([[[ 0.3704557 , -0.32115948,  0.18933958, -0.08541822,
         -0.14977457,  0.0468668 ,  0.1009687 ,  0.26232383],
        [ 0.27435896, -0.26944676,  0.1183916 , -0.09587738,
         -0.13968669,  0.03337868,  0.13802463,  0.25135165],
        [ 0.22526725, -0.24302894,  0.08214732, -0.10122051,
         -0.13453321,  0.02648817,  0.15695491,  0.24574642],
        [ 0.42170674, -0.34873927,  0.22717807, -0.07984006,
         -0.1551547 ,  0.05406038,  0.08120575,  0.26817557]],

       [[ 0.02418244,  0.23645423, -0.16074173, -0.27260572,
         -0.24574494,  0.05404392,  0.29562226,  0.18197405],
        [-0.02077094,  0.13493071, -0.16195679, -0.22687748,
         -0.19622158,  0.02884698,  0.29225895,  0.19066069],
        [ 0.02917756,  0.24773528, -0.16060668, -0.2776869 ,
         -0.25124782,  0.05684374,  0.29599592,  0.18100877],
        [ 0.05696137,  0.31048274, -0.1598557 , -0.3059496 ,
         -0.28185615,  0.

In [ ]:
# 对每一候选item，用4个向量从不同`兴趣`角度刻画user_emb；
# 之前user_emb唯一，不同的target_emb与同一兴趣user_emb计算，可能导致user_emb表达不够，兴趣互损；多个emb表示user，兴趣聚类;
# 用不同的user_part_emb取与target_emb计算损失
atten = tf.matmul(user_eb, tf.transpose(target_emb, [0,2,1]))
atten = tf.nn.softmax(tf.pow(tf.squeeze(atten,2), 1))
atten,tf.argmax(atten, axis=1, output_type=tf.int32)

(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[0.24807923, 0.25194046, 0.2539361 , 0.24604419],
        [0.25027686, 0.24217027, 0.25119427, 0.2563586 ],
        [0.25      , 0.25      , 0.25      , 0.25      ]], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 0], dtype=int32)>)

In [ ]:
readout = tf.gather(tf.reshape(user_eb, [-1, 8]), tf.argmax(atten, axis=1, output_type=tf.int32) + tf.range(tf.shape(item_list_emb)[0]) * num_heads)
readout

<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.22526725, -0.24302894,  0.08214732, -0.10122051, -0.13453321,
         0.02648817,  0.15695491,  0.24574642],
       [ 0.05696137,  0.31048274, -0.1598557 , -0.3059496 , -0.28185615,
         0.07241692,  0.2980746 ,  0.1756399 ],
       [-0.30083585,  0.35435283,  0.2086421 , -0.07965193, -0.27387637,
         0.50393224, -0.3017337 , -0.05612677]], dtype=float32)>

In [ ]:
mid_embeddings_var = tf.compat.v1.get_variable("mid_embedding_var", [2, 8], trainable=True)
mid_embeddings_bias = tf.compat.v1.get_variable("bias_lookup_table", [2], initializer=tf.zeros_initializer(), trainable=False)

def build_sampled_softmax_loss(item_emb, input_emb):
  loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(mid_embeddings_var, mid_embeddings_bias, tf.reshape(label, [-1, 1]), input_emb, 2, 2, 1))
  return loss

build_sampled_softmax_loss(target_emb, readout)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6039441>

## 聚合模块

对于在线服务，使用**多兴趣提取**来计算每个用户的多元兴趣。用户的每个兴趣向量都可以通过**Faiss**等近邻检索`top-N`物品。多个兴趣检索到的物品被输入**聚合模块**，以确定最终候选物品。

K个兴趣，每个兴趣均得到top-N，最终得到`k*N=>S`候选集集合。
$$f(u,i) = \max_{1<=k<=K}(e_i^T v_u^{(k)})$$

![](https://pic4.zhimg.com/v2-9152d31dec3f0f9eb301e7799cacecc7_b.jpg)

- 最终得到多样性得分，和兴趣拟合最大得分的item。
- 这里的，item拟合最近兴趣的最大得分，说明item，相似于某个兴趣，而且很接近即可。不同兴趣的得分值域大小并不重要，只能说明某个兴趣更加集中。
`argmax (f(u,i), + λ \sum_k g(i,k))`


参考：https://zhuanlan.zhihu.com/p/180058376